In [3]:
import pandas as pd
import numpy as np
import time
import googlemaps

In [ ]:
routes_csv = pd.read_csv('../data/routesDist.csv')

In [ ]:
usSrcRoutes = routes_csv[routes_csv['SRCCTRY'] == 'United States']
print(len(usSrcRoutes))

usDestinRoutes = routes_csv[routes_csv['DESTINCTRY'] == 'United States']
print(len(usDestinRoutes))

In [ ]:
mergeCols = ['AIRLINE_ID', 'SRC_AIRPT_ID', 'DESTIN_AIRPT_ID']

usRoutes = usSrcRoutes.merge(usDestinRoutes, how='outer', on=mergeCols)
len(usRoutes)

In [ ]:
usRoutesTest = pd.concat([usSrcRoutes,usDestinRoutes])
usRoutesTest = usRoutesTest.drop_duplicates()
len(usRoutesTest)

In [ ]:
#re-using variable above that was used to show concat and outer join provide similar data
usRoutes = usRoutesTest[['SRC_AIRPT_ID', 'SRCNAME', 'DESTIN_AIRPT_ID', 'DESTINNAME']]

In [ ]:
def distCalc(x, y):
    file = open("../dataEngineering/gMapsAPIKey.txt")
    gMapsKey = file.read()
    file.close()

    gmaps = googlemaps.Client(key=gMapsKey)

    distance = gmaps.distance_matrix(x, y)['rows'][0]['elements'][0]

    try:
        meters = distance['distance']['value']
    except:
        meters = r'\N'

    try:
        seconds = distance['duration']['value']
    except:
        seconds = r'\N'

    return(meters,seconds)

In [ ]:
usRoutes = usRoutes.reset_index()
usRoutes = usRoutes.rename(columns={'index': 'og_index'})

In [ ]:
permM = []
permS = []

In [ ]:
m = []
s = []

for i in range(len(usRoutes)):
    mVal, sVal = distCalc(usRoutes['SRCNAME'][i], usRoutes['DESTINNAME'][i])
    m.append(mVal)
    s.append(sVal)
    

In [ ]:
permM.append(m)
permS.append(s)

In [ ]:
permSeriesM = pd.DataFrame(permM)
permSeriesS = pd.DataFrame(permS)

permSeriesM.to_csv('../data/meters.csv')
permSeriesS.to_csv('../data/seconds.csv')

In [ ]:
secondsSeries = pd.Series(permSeriesS.to_numpy().flatten())
metersSeries = pd.Series(permSeriesM.to_numpy().flatten())

usRoutes["seconds"] = secondsSeries
usRoutes["meters"] = metersSeries

In [ ]:
usRoutes.to_csv('../data/usRoutes.csv')

In [ ]:
#graphs to make, plot on the map routes that can be made
#A graph that shows how many routes are in a certain distancce


In [ ]:
import plotly.graph_objects as go

airports = pd.read_csv('../data/openFlightsRaw/airports.csv')
usRoutes = pd.read_csv('../data/usRoutes.csv')

mapDat = usRoutes.merge(airports, how="left", left_on='SRCNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'SRCLAT', 'LONG': 'SRCLONG'})

mapDat = mapDat.merge(airports, how="left", left_on='DESTINNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'DESTINLAT', 'LONG': 'DESTINLONG'})

srcFig = go.Figure(data=go.Scattergeo(
    name = "Has Outgoing Traffic",
    lat = mapDat['SRCLAT'].tolist(),
    lon = mapDat['SRCLONG'].tolist(), 
    mode = 'markers',
    opacity=0.70,
    marker = dict(
        size = 5,
        color = 'blue',
        symbol = 'triangle-up',
        standoff = 3
    )
))

destinFig = go.Figure(data=go.Scattergeo(
    name = "Has Incoming Traffic",
    lat = mapDat['DESTINLAT'].tolist(),
    lon = mapDat['DESTINLONG'].tolist(), 
    mode = 'markers',
    opacity=0.70,
    marker = dict(
        size = 5,
        color = 'red',
        symbol = 'triangle-down'
    )
))

fig = go.Figure(data=go.Scattergeo())
fig.add_traces(srcFig._data)
fig.add_traces(destinFig._data)

fig.update_layout(
    title_text='All US Airports have Incoming and Outgoing Traffic',
    showlegend=True,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

In [19]:
import plotly.express as px

usRoutes = pd.read_csv('../data/usRoutes.csv')

nullVal = r"\N"
usRoutes = usRoutes[usRoutes['meters'] != nullVal]

usRoutes['meters'] = usRoutes['meters'].astype(int)
usRoutes['seconds'] = usRoutes['seconds'].astype(int)

usRoutes['hours'] = round(usRoutes['seconds']/3600, 0)

usRoutes['countVal'] = 1
usRoutesHours = usRoutes.groupby('hours')['countVal'].sum()
usRoutesHours = pd.DataFrame(usRoutesHours)
usRoutesHours = usRoutesHours.reset_index()

fig = px.bar(usRoutesHours, x='hours', y='countVal')
fig.show()

usRoutes['km'] = round(usRoutes['meters']/1000, -2)
usRoutesKM = usRoutes.groupby('km')['countVal'].sum()
usRoutesKM = pd.DataFrame(usRoutesKM)
usRoutesKM = usRoutesKM.reset_index()

fig2 = px.bar(usRoutesKM, x='km', y='countVal')
fig2.show()

In [16]:
import plotly.graph_objects as go

airports = pd.read_csv('../data/openFlightsRaw/airports.csv')
usRoutes = pd.read_csv('../data/usRoutes.csv')

mapDat = usRoutes.merge(airports, how="left", left_on='SRCNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'SRCLAT', 'LONG': 'SRCLONG'})

mapDat = mapDat.merge(airports, how="left", left_on='DESTINNAME', right_on='NAME')
mapDat = mapDat.rename(columns={'LAT': 'DESTINLAT', 'LONG': 'DESTINLONG'})

nullVal = r"\N"
mapDat = mapDat[mapDat['meters'] != nullVal]

fig = go.Figure()

for index, row in mapDat.iterrows():

    fig.add_trace(go.Scattergeo(
        mode = "markers+lines",
        lat = [row['SRCLAT'], row['DESTINLAT']],
        lon = [row['SRCLONG'], row['DESTINLONG']],
        opacity=0.01,
        marker = dict(
            size = 10,
            color = 'black'
    )
        ))

fig.update_layout(
    title_text='All US Airports have Incoming and Outgoing Traffic',
    showlegend=False,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

In [23]:
usRoutes = pd.read_csv('../data/usRoutes.csv')

nullVal = r"\N"
usRoutes = usRoutes[usRoutes['meters'] == nullVal]

usRoutes['countVal'] = 1
usRoutesAirlines = usRoutes.groupby('SRCNAME')['countVal'].sum()
usRoutesAirlines = pd.DataFrame(usRoutesAirlines)
usRoutesAirlines = usRoutesAirlines.reset_index()

usRoutesAirlines = usRoutesAirlines[usRoutesAirlines['countVal'] >= 50]
fig = px.bar(usRoutesAirlines, x='SRCNAME', y='countVal')
fig.show()